In [8]:
import pandas as pd

# Load your CSV file (after downloading from Box)
df = pd.read_csv('train.csv')  # replace with actual filename



In [11]:

# Check missing values
print("Missing values of each coloum:\n",df.isnull().sum())

# Remove non-numeric characters using regex and convert to float
df['Mileage'] = df['Mileage'].astype(str).str.extract(r'(\d+\.?\d*)').astype(float)
df['Engine'] = df['Engine'].astype(str).str.extract(r'(\d+\.?\d*)').astype(float)
df['Power'] = df['Power'].astype(str).str.extract(r'(\d+\.?\d*)').astype(float)

df['Mileage'] = df['Mileage'].fillna(df['Mileage'].median())
df['Engine'] = df['Engine'].fillna(df['Engine'].median())
df['Power'] = df['Power'].fillna(df['Power'].median())
df['Seats'] = df['Seats'].fillna(df['Seats'].mode()[0])


# Drop rows with critical missing values if necessary
df = df.dropna(subset=['Price'])  # example

Missing values of each coloum:
 Unnamed: 0              0
Name                    0
Location                0
Year                    0
Kilometers_Driven       0
Fuel_Type               0
Transmission            0
Owner_Type              0
Mileage                 0
Engine                  0
Power                  36
Seats                  38
New_Price            5032
Price                   0
dtype: int64


In [19]:
import pandas as pd

# Load the raw dataset
df = pd.read_csv('train.csv')  # Replace with your filename

# Step 1: Convert to string for regex unit removal
for col in ['Mileage', 'Engine', 'Power', 'New_Price']:
    df[col] = df[col].astype(str)

# Step 2: Remove units from text using regex
df['Mileage'] = df['Mileage'].str.extract(r'(\d+\.?\d*)').astype(float)
df['Engine'] = df['Engine'].str.extract(r'(\d+\.?\d*)').astype(float)
df['Power'] = df['Power'].str.extract(r'(\d+\.?\d*)').astype(float)
df['New_Price'] = df['New_Price'].str.replace(' Lakh', '', regex=False)
df['New_Price'] = df['New_Price'].str.extract(r'(\d+\.?\d*)').astype(float)

# Step 3: Handle missing values
df['Mileage'] = df['Mileage'].fillna(df['Mileage'].median())
df['Engine'] = df['Engine'].fillna(df['Engine'].median())
df['Power'] = df['Power'].fillna(df['Power'].median())
df['Seats'] = df['Seats'].fillna(df['Seats'].mode()[0])

# Drop rows with missing target variable (Price)
df = df.dropna(subset=['Price'])

# Step 4: Export the cleaned dataset
df.to_csv('cleaned_used_cars.csv', index=False)

print("✅ Cleaned dataset saved as 'cleaned_used_cars.csv'")


✅ Cleaned dataset saved as 'cleaned_used_cars.csv'


In [21]:
import pandas as pd

# Load the cleaned dataset from the previous step
df = pd.read_csv('cleaned_used_cars.csv')  # This is your already cleaned file (missing values handled)

# Step: Convert to string to perform unit removal
df['Mileage'] = df['Mileage'].astype(str).str.extract(r'(\d+\.?\d*)').astype(float)
df['Engine'] = df['Engine'].astype(str).str.extract(r'(\d+\.?\d*)').astype(float)
df['Power'] = df['Power'].astype(str).str.extract(r'(\d+\.?\d*)').astype(float)
df['New_Price'] = df['New_Price'].astype(str).str.replace(' Lakh', '', regex=False)
df['New_Price'] = df['New_Price'].str.extract(r'(\d+\.?\d*)').astype(float)

# Save the fully cleaned data to a new file
df.to_csv('fully_cleaned_used_cars.csv', index=False)

print("✅ Units removed and dataset saved as 'fully_cleaned_used_cars.csv'")


✅ Units removed and dataset saved as 'fully_cleaned_used_cars.csv'


In [22]:
import pandas as pd

# Load the cleaned data (with units removed)
df = pd.read_csv('fully_cleaned_used_cars.csv')

# Perform one-hot encoding for 'Fuel_Type' and 'Transmission'
df_encoded = pd.get_dummies(df, columns=['Fuel_Type', 'Transmission'], drop_first=True)

# Save the final encoded dataset
df_encoded.to_csv('encoded_used_cars.csv', index=False)

print("✅ One-hot encoded dataset saved as 'encoded_used_cars.csv'")


✅ One-hot encoded dataset saved as 'encoded_used_cars.csv'


In [23]:
import pandas as pd
from datetime import datetime

# Load the one-hot encoded dataset
df = pd.read_csv('encoded_used_cars.csv')

# Calculate current year
current_year = datetime.now().year

# Create new feature: Car Age
df['Car_Age'] = current_year - df['Year']

# Save updated dataset
df.to_csv('final_used_cars_with_age.csv', index=False)

print("✅ 'Car_Age' column added. Saved as 'final_used_cars_with_age.csv'")


✅ 'Car_Age' column added. Saved as 'final_used_cars_with_age.csv'


In [30]:
import pandas as pd
from datetime import datetime

# ✅ Load your data
df = pd.read_csv('final_used_cars_with_age.csv')

# ✅ Add Car_Age if not present
if 'Car_Age' not in df.columns:
    df['Car_Age'] = datetime.now().year - df['Year']

# ✅ One-hot encode if not already encoded
if 'Fuel_Type' in df.columns or 'Transmission' in df.columns:
    df = pd.get_dummies(df, columns=['Fuel_Type', 'Transmission'], drop_first=True)

# ✅ Define safe selections
columns_to_select = ['Name', 'Year', 'Price', 'Car_Age']
if 'Fuel_Type_Diesel' in df.columns:
    columns_to_select.append('Fuel_Type_Diesel')
if 'Transmission_Automatic' in df.columns:
    columns_to_select.append('Transmission_Automatic')

# ✅ Select
selected_df = df[columns_to_select]

# ✅ Filter: cars older than 10 years
filtered_df = selected_df[selected_df['Car_Age'] > 10]

# ✅ Rename columns (if exist)
df = df.rename(columns={
    'Kilometers_Driven': 'Odometer',
    'Power': 'Horsepower',
    'New_Price': 'Original_Price'
})

# ✅ Mutate: calculate km per year
df['Km_per_Year'] = df['Odometer'] / df['Car_Age'].replace(0, 1)

# ✅ Arrange: sort by Price
sorted_df = df.sort_values(by='Price', ascending=False)

# ✅ Summarize: average price by fuel type
if 'Fuel_Type_Diesel' in df.columns:
    summary_df = df.groupby('Fuel_Type_Diesel')['Price'].mean().reset_index().rename(
        columns={'Fuel_Type_Diesel': 'Is_Diesel', 'Price': 'Avg_Price'}
    )
else:
    summary_df = pd.DataFrame({'Is_Diesel': ['Column Missing'], 'Avg_Price': [0]})

# ✅ Save outputs
selected_df.to_csv('selected_columns.csv', index=False)
filtered_df.to_csv('filtered_cars_over_10_years.csv', index=False)
sorted_df.to_csv('sorted_by_price.csv', index=False)
summary_df.to_csv('avg_price_by_fuel_type.csv', index=False)

# ✅ Show summary
print("✅ Summary - Avg Price by Fuel Type (Diesel=1):")
print(summary_df)


✅ Summary - Avg Price by Fuel Type (Diesel=1):
        Is_Diesel  Avg_Price
0  Column Missing          0
